<a href="https://colab.research.google.com/github/novouss/fake-news-detection/blob/main/fake_news_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/novouss/fake-news-detection.git

fatal: destination path 'fake-news-detection' already exists and is not an empty directory.


In [2]:
!ls

fake-news-detection  sample_data


# Preliminary Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import cohen_kappa_score

import re
import nltk

nltk.download('stopwords')

from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Declaration of functions

## Pie Chart Generator

In [4]:
def getPie(true: list, false: list):
    d = [len(true), len(false)]
    t = ['Real', 'Fake']

    plt.pie(d, labels = t, autopct = '%1.2f%%')
    plt.show()

## WorldCloud Generator

In [5]:
def getWordCloud(corpus: list):
    text = ' '
    
    for word in corpus:
        text += ' '.join(word.spit(' '))

    word_cloud = WordCloud(collocations = False, background_color = 'white').generate(text)
    plt.imshow(word_cloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

## Feature Extraction

In [6]:
def getFeature(input: list):

    corpus = []

    for i in range(0, 40000):
        arr = re.sub('[^a-zA-Z]', ' ', input[i])
        arr = arr.lower()
        arr = arr.split()

        ps = PorterStemmer()

        arr = [ps.stem(word) for word in arr if not word in set(stopwords.words('english'))] 
        arr = ' '.join(arr)

        corpus.append(arr)
    
    return corpus

## Print Accuracy

In [7]:
def accuracy(title, x):
    print('-' * 50, '\n' + title, 'Accuracy: %.2f%%' % x)

# Declaration of Validation functions


## Holdout Validation Approach

In [8]:
def holdout(X, Y):
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
    model = LogisticRegression()
    model.fit(X_train, Y_train)
    result = model.score(X_train, Y_train)
    
    accuracy('Holdout Validation Approach', (result * 100))

## K-fold Cross-Validation

In [9]:
def kfold(X, Y):
    kfold = KFold(n_splits = 5)
    model = LogisticRegression()
    results = cross_val_score(model, X, Y, cv = kfold)

    accuracy('K-fold Cross-Validation', (results.mean() * 100))

## ShuffleSplit Validation

In [10]:
def sufflesplit(X, Y):
    kfold2 = ShuffleSplit(n_splits = 10, test_size = 0.20)
    model = LogisticRegression()
    results = cross_val_score(model, X, Y, cv = kfold2)

    accuracy('Repeated Random Test-Train Splits', (results.mean() * 100))

# Data processing

In [11]:
false_dataset = pd.read_csv('//content//fake-news-detection//datasets//fake.csv')
true_dataset = pd.read_csv('//content//fake-news-detection//datasets//true.csv')

In [12]:
false_dataset.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [13]:
true_dataset.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [14]:
false_dataset['fake'] = 0
true_dataset['fake'] = 1

table = pd.DataFrame()
table = true_dataset.append(false_dataset)
table = table.drop(columns=['subject','date'])

In [15]:
print('-' * 50)
print("Table shape:", table.shape)

--------------------------------------------------
Table shape: (44898, 3)


# Modeling

In [16]:
input_arr = np.array(table['title'])
corpus = getFeature(input_arr)
countv = CountVectorizer(max_features = 5000)

X = countv.fit_transform(corpus).toarray()
Y = table.iloc[0:40000, 2].values

In [17]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)
model = LogisticRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

print('-' * 50)
print('Classification report\n', classification_report(Y_test, Y_pred))
print('Confusion Matrix\n', confusion_matrix(Y_test, Y_pred))
print('Precision: ', precision_score(Y_test, Y_pred))
print('Accuracy: ', accuracy_score(Y_test, Y_pred))
print('Cohen Kappa: ', cohen_kappa_score(Y_test, Y_pred))

--------------------------------------------------
Classification report
               precision    recall  f1-score   support

           0       0.96      0.93      0.95      3752
           1       0.94      0.97      0.95      4248

    accuracy                           0.95      8000
   macro avg       0.95      0.95      0.95      8000
weighted avg       0.95      0.95      0.95      8000

Confusion Matrix
 [[3488  264]
 [ 140 4108]]
Precision:  0.939615736505032
Accuracy:  0.9495
Cohen Kappa:  0.8984142566035762


# Validation

In [18]:
holdout(X, Y)

-------------------------------------------------- 
Holdout Validation Approach Accuracy: 97.45%


In [19]:
kfold(X, Y)

-------------------------------------------------- 
K-fold Cross-Validation Accuracy: 87.15%


In [20]:
sufflesplit(X, Y)

-------------------------------------------------- 
Repeated Random Test-Train Splits Accuracy: 94.82%


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
